<a href="https://colab.research.google.com/github/Vlastellin/HandTracking/blob/master/nn_train2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [14]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
import pickle
import os
import imageio
import numpy as np
from tensorflow.keras.preprocessing import image

def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  #print(img.shape)
  img = np.expand_dims(img, axis=0)
  #print(img.shape)
  #img = inception_v3.preprocess_input(img)
  return img

base_dir = '/content/drive/My Drive/find_3d_points'

input_img = Input(shape=(480,640, 3))
x = layers.Conv2D(32, (3, 3), activation='relu')(input_img)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512)(x)
p1 = layers.Dense(3, name='p1')(x)
p2 = layers.Dense(3, name='p2')(x)
p3 = layers.Dense(3, name='p3')(x)
model = Model([input_img],[p1, p2, p3])
model.compile(optimizer='rmsprop',loss=['mae', 'mae', 'mae'])
#data
imgs=[]
koords=[[],[],[]]
with open(os.path.join(base_dir, 'dataset_list.pickle'), 'rb') as f:
  l= pickle.load(f)
for i,koord in enumerate(l[1]):
  img_path=os.path.join(base_dir, 'dataset/'+str(i)+'.jpg')
  #image_raw = imageio.imread(img_path)
  #image_v = np.expand_dims((image_raw.astype('float') / 255.0) - 0.5, 0)
  imgs.append(preprocess_image(img_path)[0])
  #print(image_raw.shape)
  temp_koords=np.array([np.array([koord[0][0], koord[0][1],koord[0][2]]), np.array([koord[1][0], koord[1][1],koord[1][2]]),np.array([koord[2][0], koord[2][1],koord[2][2]])])
  for j,k in enumerate(temp_koords):
    koords[j].append(k)
  if i==150:
    break

#for k in koords:
# k= np.array(k)
#print(koords[0])
imgs = np.array(imgs)
print(imgs.shape)
print(imgs.ndim)
koords=np.array([koords[0], koords[1], koords[2]])
print(type(koords[0]))
#print(np.array(koords))
print(koords.shape)
from tensorflow.keras.utils import plot_model
#plot_model(model, to_file=os.path.join(base_dir,'model.png'), show_shapes=True)
#model.fit(x=imgs, y={'p0_x':koords[0], 'p0_y':koords[1], 'p0_z':koords[2],'p5_x':koords[3], 'p5_y':koords[4], 'p5_z':koords[5],'p9_x':koords[6], 'p9_y':koords[7], 'p9_z':koords[8]},epochs=2, batch_size=64)
model.fit(x=(imgs), y={'p1':koords[0], 'p2':koords[1], 'p3':koords[2]},epochs=20, validation_split=0.1, batch_size=150)

(151, 480, 640, 3)
4
<class 'numpy.ndarray'>
(3, 151, 3)
Train on 135 samples, validate on 16 samples
Epoch 1/20
135/135 [==============================] - 57s 419ms/sample - loss: 240.5696 - p1_loss: 110.3859 - p2_loss: 73.9776 - p3_loss: 56.2060 - val_loss: 53715.4180 - val_p1_loss: 20414.3828 - val_p2_loss: 14398.1553 - val_p3_loss: 18902.8789
Epoch 2/20
135/135 [==============================] - 54s 401ms/sample - loss: 53491.4062 - p1_loss: 20328.6855 - p2_loss: 14339.7373 - p3_loss: 18822.9824 - val_loss: 185.0320 - val_p1_loss: 46.9542 - val_p2_loss: 65.0838 - val_p3_loss: 72.9940
Epoch 3/20
135/135 [==============================] - 53s 396ms/sample - loss: 168.3425 - p1_loss: 44.2501 - p2_loss: 57.5834 - p3_loss: 66.5091 - val_loss: 3142.6099 - val_p1_loss: 1328.3875 - val_p2_loss: 619.9129 - val_p3_loss: 1194.3097
Epoch 4/20
135/135 [==============================] - 54s 400ms/sample - loss: 3140.8962 - p1_loss: 1325.9534 - p2_loss: 621.4142 - p3_loss: 1193.5287 - val_loss: 4

In [0]:
!pip install -U numpy


Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
